# IESB
# Graduação em Ciência de Dados
## Disciplina - Mineração de Dados e Inteligência Computacional
## Avaliação - 04/05/2020

### Descrição
A base de dados "Home Equity" possui dados pessoas e informações de empréstimo de 5.960 empréstimos recentes. Para cada empréstimo existem 12 variáveis registradas.
A variável alvo (`BAD`) indica quando o cliente não pagou o empréstimo (valor 1), e quando ele honrou o compromisso (valor 0).

### Objetivo
Criar um modelo de ML usando RandomForest para prever se determinado cliente irá ou não honrar o empréstimo contraído.

O desempenho do modelo deve ser medido pela métrica de acurácia (`accuracy_score`).

In [ ]:
# Arquivo de dados
import os
print(os.listdir("../input"))

**Itens avaliados:**

1. Carregamento dos dados
2. Verificação para saber se os dados foram importados corretamente
3. Pré-processamento dos dados (preenchimento de valores em branco, transformação de texto em número)
4. Separar dados em treino e validação
5. Instanciar e treinar uma `RandomForest`
6. Avaliação da performance do modelo pela métrica de acurácia (`accuracy_score`).
7. Buscar a melhoria da acurácia mudando os parâmetros do modelo RandomForest.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn_pandas import CategoricalImputer
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/hmeq.csv")
df.head(5)

## Questão 1
- Dados carregados

In [ ]:
df.info()

## Questão 2
- Os dados foram importados corretamente, os tipos estão corretos

In [ ]:
df.isnull().sum().sort_values()

In [ ]:
df.JOB.value_counts(dropna=False)

In [ ]:
# Para os casos NAN de job vamos considerar desempregado
df.JOB = df.JOB.fillna("Unemployed")

In [ ]:
df.REASON.value_counts(dropna=False)

In [ ]:
#Para os casos NAN vamos considerar que não se sabe, "Unknown"
df.REASON = df.REASON.fillna("Unknown")

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
df.hist(bins=30, figsize=(15, 10))
plt.show()

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
df_dum = pd.get_dummies(df, columns=["REASON","JOB"], drop_first=True)

In [ ]:
df_dum.head(5)

## Questão 3
- Foi aplicado fillna 0 nas colunas número e fillna(string) apropriada para cada caso de coluna objeto
- Foi feito o getdummies apropriado

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV

X, y = df_dum.drop("BAD",axis=1),df_dum["BAD"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train, test_size=0.2, random_state=42)


rfc = RFC(n_estimators=100)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_valid)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0),"\n")
conf_mat = confusion_matrix(y_true=y_valid, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)
print('\nClassification Report:\n',classification_report(y_valid, y_pred))

In [ ]:
n_estimators = [100, 110,120]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5]
min_samples_leaf = [1, 2, 5] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(RFC(), hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, y_train)

In [ ]:
bestF.best_params_

In [ ]:
rfc = RFC(max_depth= 25,
            min_samples_leaf= 1,
            min_samples_split= 2,
            n_estimators= 120)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0),"\n")
conf_mat = confusion_matrix(y_true=y_valid, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)
print('\nClassification Report:\n',classification_report(y_valid, y_pred))

Depois do tuning de hyperameter a perforamnce de nosso modelo caiu, preferiu-se deixar a padrão, porém n_estimators = 100

# Bônus

Após terminar o exercício e verificar que está cumprindo todos os itens a serem avaliados, você pode fazer os itens a seguir para ganhar pontos extra na prova:

**Bônus 1:**
    
Plote uma matriz de confusão nas previsões dos dados de validação.

In [ ]:
print('Confusion matrix:\n', conf_mat)

**Bônus 2:**
    
Explique o que é `precisão` na matriz de confusão acima e dê 1 exemplo onde isso pode ser útil.

- Precisão avalia quanto dos casos que não previmos como positive na verdade são positive. Isso pode ser útil quando devemos priorizar não ter um erro em meio do que dizemos ser positivo.
- Por exemplo nós não queremos errar o lançamento de foguete para outra planeta. Nossa rota prevista tem que estar correta.
- ou no caso acima, se for priorizada a precisão, isso signfica que quando o cara é mal pagante, nós queremos ter mais certeza possível de que não será tomada medidas drásticas contra a pessoa errada. Ou seja, nós somos mais abertos a mal pagantes adentrarem em nosso esquema.

**Bônus 3:**
    
Explique o que é `recall` na matriz de confusão acima e dê 1 exemplo onde isso pode ser útil.

- Recall avalia quanto dos casos que deveriamos prever como positive não previmos.
- No caso do modelo acima, avaliaria quantos porcentos de não pagantes previmos dentre todos os pagantes.
- Essa métrica pode ser priorizada, quando queremos ter certeza que englobamos todos os não pagantes, e não fazemos tanta questão de confundir um bom pagante por um mal pagante. Ou seja temos politica zero de abertura a mal pagantes.